In [40]:
import numpy as np
from pymatgen.core import Structure, PeriodicSite, DummySpecie
# from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import MinimumDistanceNN

In [ ]:
'''# Load defective structure
defective_struct = Structure.from_file("low_density_defects/MoS2/MoS2cifs/6141cf1a1d648932fbc34297.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("low_density_defects/MoS2/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])'''


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 48 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [41]:
# Load defective structure
defective_struct = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/highMoS2cifs/MoS2_Mo56W4Se3S117_b593cd47-47e7-4a55-9a27-466f5c3f661c.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 45 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [42]:
def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict

In [ ]:
def get_defects(defective_struct, reference_struct):
    copy_defective_struct = defective_struct.copy()
    # struct to dict
    defective_dict = struct_to_dict(copy_defective_struct)
    reference_dict = struct_to_dict(reference_struct)

    # Get lattice of defective structure
    structure_lattice = copy_defective_struct.lattice

    # Define list to add all defect sites
    defects_list = []

    # Define a dictionary to hold properties
    sites_properties = {} # defect + nondefect sites
    defects_properties = {} # defect sites only

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie: # Substitution case
                    # Add site to defects list
                    defects_list.append(def_site)

                    # Get atomic number change and defect type
                    add_property = {"orignal_an":ref_site.specie.Z,
                                    "new_an": def_site.specie.Z,
                                    "an_change": def_site.specie.Z - ref_site.specie.Z,
                                    "vacancy_defect": 0.0,
                                    "substitution_defect": 1.0}
                    defects_properties[def_site] = add_property
                    sites_properties[def_site] = add_property
                else:
                    add_property = {"original_an":ref_site.specie.Z, 
                                    "new_an": def_site.specie.Z,
                                    "an_change": def_site.specie.Z - ref_site.specie.Z,
                                    "vacancy_defect": 0.0,
                                    "substitution_defect": 0.0}
                    sites_properties[def_site] = add_property

                    

        if not matching: # Vacancy case
            # Add site to defective structure
            vacant_site = PeriodicSite(
                species= DummySpecie(),
                coords= ref_coords,
                coords_are_cartesian= False, 
                lattice= structure_lattice
                )
            
            copy_defective_struct.append(vacant_site.species, vacant_site.frac_coords)

            # Add site to defects list
            defects_list.append(vacant_site)

            # Get atomic number change and defect type
            add_property={"original_an":ref_site.specie.Z,
                          "new_an": 0,
                          "an_change": 0 - ref_site.specie.Z,
                          "vacancy_defect": 1.0,
                          "substitution_defect": 0.0}
            defects_properties[vacant_site] = add_property
            sites_properties[vacant_site] = add_property

    return copy_defective_struct, sites_properties, defects_list, defects_properties


# Get defect structure
new_defective_struct, new_defective_struct_properties, defect_sites, defect_properties = get_defects(defective_struct, reference_struct)


In [ ]:
def prop_in_struct(properties, structure):
    # Create copy of structure
    copy_structure = structure.copy()

    # Add properties to structure
    for a_site in copy_structure.sites:
        if a_site in properties.keys():
            a_site.properties.update(properties[a_site])
        else:
            pass

    return copy_structure

In [ ]:
def_struct = Structure.from_sites(defect_sites)

def_prop_struct = prop_in_struct(defect_properties, def_struct)
print(def_prop_struct)

In [ ]:
def get_nodes(structure):
    mdnn = MinimumDistanceNN
    nodes = []
    for i, site in enumerate(structure.sites):
        node_features = [i, site.properties["original_an"], site.properties["new_an"],
                         site.properties["an_change"], site.properties["vacancy_defect"],
                         site.properties["substitution_defect"]]
        nodes.append(node_features)

In [ ]:
def get_edges(structure):
    sites_list = structure.sites

    edges = []
    edge_features = []

    home_site = []
    away_site = []

    for i, site in enumerate(sites_list):
        props = []
        for j, site  in enumerate(sites_list):
            home_site.append(i)
            away_site.append(j)
            
            dist = site.distance(j)
            props.append(dist)

    edges.append(home_site)
    edges.append(away_site)
    edge_features.append(props)

    


In [46]:
def get_properties(new_defective_struct, all_defect_sites):
    list_of_sites = new_defective_struct.sites

    # Define an instance responsible for getting the nearest neighbors
    cnn = MinimumDistanceNN(cutoff=3.0)

    # dictionary of properties
    site_dict = {} # Of each site
    new_dict = {}  # Of all sites

    # Lets get the properties for each defect site
    for i in list_of_sites:
        the_index = list_of_sites.index(i)

        # Get the nearest neighbors of each defect site
        neighbors = cnn.get_nn_info(new_defective_struct, the_index)

        # Get the coordination numbers
        coord_numbers = len(neighbors)

        # Get the vital attributes:
        # neighbors
        the_neighbors = []
        # site index of neighbor
        neighbors_site_index = []
        # neighbors image
        neighbors_image = []
        # distance to neighbors
        bl_to_neighbors = []
        # weights to neighbors
        weights_to_neighbors = []
        # atomic numbers of neighbors
        elements_an = []

        for neighbor_info in neighbors:
            the_neighbors.append(neighbor_info['site'])
            neighbors_site_index.append(neighbor_info['site_index'])
            neighbors_image.append(neighbor_info['image'])
            bond_length = neighbor_info['site'].distance(i)
            bl_to_neighbors.append(bond_length)
            weights_to_neighbors.append(neighbor_info['weight'])
            # Handle the atomic numbers
            the_an = neighbor_info["site"].specie.Z
            if the_an > 0 and the_an < 119:
                elements_an.append(the_an)
            else:
                the_an = 0
                elements_an.append(the_an)
                
        # Get distance to other defect sites
        dist_to_other_defects = []
        for n in all_defect_sites:
            dist = i.distance(n)
            dist_to_other_defects.append(dist)

        # Collect these properties as dictionaries
        site_dict[i] = {"nearest_neighbors": the_neighbors,
                        "neighbors_atomic_numbers": elements_an,
                        "nearest_neighbors_index": neighbors_site_index,
                        "neighbors_image": neighbors_image,
                        "neighbors_weight": weights_to_neighbors,
                        "distance_to_neighbors": bl_to_neighbors,
                        "coordination_number": coord_numbers,
                        "distance_to_defects": dist_to_other_defects}
        
        new_dict.update(site_dict)
    
    return site_dict


In [ ]:
sites_properties = get_properties(new_defective_struct, all_defects)

for key1 in sites_properties.keys():
    if key1 in defect_an_property.keys():
        sites_properties[key1].update(defect_an_property[key1])
    else:
        pass

for site_struct in new_defective_struct.sites:
    if site_struct in sites_properties.keys():
        site_struct.properties.update(sites_properties[site_struct])
    else:
        pass

print(new_defective_struct)